In [1]:
# remove edges with low counts/occurence, or no good mapping, or not useful

In [1]:
import os
import pickle
%pylab
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
from collections import defaultdict
from tqdm import tqdm

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
edges = pd.read_csv('edges_sanitized.csv')

In [3]:
nodes = pd.read_csv("nodes_sanitized.csv", index_col=0)
nodes.head()

,ID,label,TYPE
0,C0038195,State Medicine,Activities & Behaviors
1,C1290952,Taking medication,Activities & Behaviors
2,C0680552,legalization,Activities & Behaviors
3,C0085092,Parenting behavior,Activities & Behaviors
4,C1096771,Murderer,Activities & Behaviors


In [4]:
abv = pd.read_csv("abv.csv")
name_abv = dict(zip(abv.full_name, abv.abv))
abv_name = dict(zip(abv.abv, abv.full_name))

In [5]:
edges.head()

,START_ID,END_ID,TYPE,pmids,n_pmids,NEG,DOMAIN,PRED,RANGE
0,C1273870,C0282623,ABatAB,17170614,1,False,AB,at,AB
1,C0935546,C0221192,ABatCI,15856663,1,False,AB,at,CI
2,C1273870,C1138603,ABatCI,12943031;15842188,2,False,AB,at,CI
3,C1273870,C0221192,ABatCI,25503364;22250100;24417454;16723949;19337198;2...,53,False,AB,at,CI
4,C0556656,C0221192,ABatCI,17684606;16230038;17323301;14707659;17530066;1...,7,False,AB,at,CI


In [6]:
vc = edges.PRED.value_counts()
vc.index = vc.index.map(lambda x:abv_name.get(x))
vc

LOCATION_OF         2784593
AFFECTS             1811542
PART_OF             1783262
INTERACTS_WITH      1777938
COEXISTS_WITH       1720576
TREATS              1461120
STIMULATES          1022236
PROCESS_OF           930313
CAUSES               895512
INHIBITS             836695
ASSOCIATED_WITH      801413
AUGMENTS             703490
USES                 616047
PRODUCES             539846
DISRUPTS             508487
compared_with        428485
ISA                  395398
DIAGNOSES            271145
PREDISPOSES          252842
METHOD_OF            207813
PREVENTS             187540
ADMINISTERED_TO      187036
higher_than          130312
PRECEDES             101478
OCCURS_IN             67383
CONVERTS_TO           57504
MANIFESTATION_OF      41592
COMPLICATES           37497
same_as               26740
lower_than            25684
different_from         6864
different_than         1729
Name: PRED, dtype: int64

In [7]:
def drop_edges(df, drop_edges):
    idx = df.query('PRED in @drop_edges').index
    df.drop(idx, inplace=True)

In [8]:
# remove edges we don't need
print(len(edges))
remove_edges = ['compared_with', 'higher_than', 'lower_than', 'different_from', 'different_than', 'same_as',
               'OCCURS_IN', 'PROCESS_OF', 'DIAGNOSES', 'METHOD_OF', 'USES',
               'AUGMENTS', 'ADMINISTERED_TO', 'COMPLICATES']
drop_edges(edges, list(map(name_abv.get, remove_edges)))
print(len(edges))

20620112
16979574


In [9]:
edge_type_counts = edges.TYPE.value_counts()
print(edge_type_counts.head())
print(edge_type_counts.tail())

CDiwCD    1047376
AloCD      881943
DOcwDO     636149
CDcwCD     556105
CDstCD     535258
Name: TYPE, dtype: int64
PHiGA     1
DVinPS    1
PHawCI    1
GtAB      1
CDpcA     1
Name: TYPE, dtype: int64


In [10]:
# somewhat arbitrarily remove type->pred->type edges with less than 1000 instances
print(len(edge_type_counts))
idx = edge_type_counts[edge_type_counts>1000]
print("keeping {} edges out of {}".format(len(idx), len(edge_type_counts)))
orig_len = len(edges)
edges = edges[edges.TYPE.isin(idx.index)]
print("keeping {} out of {} rows".format(len(edges), orig_len))

1492
keeping 259 edges out of 1492
keeping 16870659 out of 16979574 rows


In [11]:
edge_type_counts = edges.TYPE.value_counts()

In [13]:
edge_type_counts[edge_type_counts.index.str.contains("cw")]

DOcwDO    636149
CDcwCD    556105
CDcwG     268773
GcwG      119630
PScwPS     47332
PRcwPR     28407
PScwDO     12941
GcwCD       8907
LBcwDO      7216
CIcwDO      5256
LBcwLB      3612
OBcwCD      2997
PHcwDO      2667
PRcwDO      2469
ABcwPR      2116
PRcwCD      1828
GcwA        1798
ABcwDO      1255
PScwPH      1054
Name: TYPE, dtype: int64

In [14]:
print(len(nodes))

268917


In [15]:
# remove nodes with no edges
nodes = nodes[nodes.ID.isin(set(list(edges.START_ID) + list(edges.END_ID)))]

In [16]:
print(len(nodes))

254222


In [17]:
vc = edges.PRED.value_counts()
vc.index = vc.index.map(lambda x:abv_name.get(x))
vc

LOCATION_OF         2775247
AFFECTS             1801719
PART_OF             1775266
INTERACTS_WITH      1769808
COEXISTS_WITH       1710512
TREATS              1451933
STIMULATES          1016496
CAUSES               891869
INHIBITS             832129
ASSOCIATED_WITH      794624
PRODUCES             534271
DISRUPTS             503352
ISA                  387776
PREDISPOSES          250041
PREVENTS             182436
PRECEDES              97182
CONVERTS_TO           57223
MANIFESTATION_OF      38775
Name: PRED, dtype: int64

In [18]:
edges.to_csv('edges_filtered.csv', index=False)
nodes.to_csv("nodes_filtered.csv", index=False)